In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
nhl_2018 = nhl_df[nhl_df['year'] > 2017]
cities_nhl = cities.filter(items=['Metropolitan area', 'Population (2016 est.)[8]', 'NHL'])
cities_nhl['NHL'].replace({'—':np.nan, '\[note\s[0-9]+\]':' '}, regex=True, inplace=True)
cities_nhl['NHL'].replace({'^\s':np.nan, '\s':''}, regex=True, inplace=True)
city_2 = cities_nhl.dropna()
city_2.set_index('Metropolitan area', inplace=True)
city_2.sort_index(inplace=True)
city_2['Population (2016 est.)[8]']=city_2['Population (2016 est.)[8]'].astype(int)
nhl_2018['Metropolitan area']=nhl_2018['team']
nhl_2018['Metropolitan area'].replace({'Tampa Bay Lightning*':'Tampa Bay Area', 'Boston Bruins*':'Boston', 'Toronto Maple Leafs*':'Toronto', 'Florida Panthers':'Miami–Fort Lauderdale', 'Detroit Red Wings':'Detroit', 'Montreal Canadiens':'Montreal', 'Ottawa Senators':'Ottawa', 'Buffalo Sabres':'Buffalo','Washington Capitals*':'Washington, D.C.', 'Pittsburgh Penguins*':'Pittsburgh', 'Philadelphia Flyers*':'Philadelphia', 'Columbus Blue Jackets*':'Columbus', 'New Jersey Devils*':'New York City', 'Carolina Hurricanes':'Raleigh', 'New York Islanders':'New York City', 'New York Rangers':'New York City', 'Nashville Predators*':'Nashville', 'Winnipeg Jets*':'Winnipeg', 'Minnesota Wild*':'Minneapolis–Saint Paul', 'Colorado Avalanche*':'Denver', 'St. Louis Blues':'St. Louis', 'Dallas Stars':'Dallas–Fort Worth', 'Chicago Blackhawks':'Chicago','Vegas Golden Knights*':'Las Vegas', 'Anaheim Ducks*':'Los Angeles', 'San Jose Sharks*':'San Francisco Bay Area', 'Los Angeles Kings*':'Los Angeles', 'Calgary Flames':'Calgary', 'Edmonton Oilers':'Edmonton', 'Vancouver Canucks':'Vancouver', 'Arizona Coyotes':'Phoenix'}, inplace=True)
nhl_2018.replace('\w*\sDivision',np.nan, regex=True, inplace=True)
nhl = nhl_2018.dropna().filter(items=['W', 'L', 'Metropolitan area'])
nhl[['W', 'L']] = nhl[['W', 'L']].astype(int)
nhl_2= nhl.groupby('Metropolitan area').agg({'W':np.mean, 'L':np.mean})

def wlr():
    global nhl_2
    nhl_2['W/L ratio'] = nhl_2['W']/(nhl_2['W']+nhl_2['L'])
    return nhl_2
nhl_3 = wlr()
def nhl_correlation(): 
    
    population_by_region = list(city_2['Population (2016 est.)[8]'])
    win_loss_by_region = list(nhl_3['W/L ratio'])

    assert len(population_by_region) == len(win_loss_by_region)
    assert len(population_by_region) == 28
    
    answer = stats.pearsonr(population_by_region, win_loss_by_region)
    return answer[0]
print(nhl_correlation())

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities_nba = cities.filter(items=['Metropolitan area', 'Population (2016 est.)[8]', 'NBA'])
cities_nba['NBA'].replace({'—':np.nan, '\[note\s[0-9]+\]':' '}, regex=True, inplace=True)
cities_nba['NBA'].replace({'^\s':np.nan, '\s':''}, regex=True, inplace=True)
city_2 = cities_nba.dropna()
city_2.set_index('Metropolitan area', inplace=True)
city_2.sort_index(inplace=True)
city_2['Population (2016 est.)[8]']=city_2['Population (2016 est.)[8]'].astype(float)

nba_2018 = nba_df[nba_df['year'] > 2017]
nba_2018['Metropolitan area']=nba_2018['team']
nba_2018['Metropolitan area'].replace({'Toronto Raptors*\xa0(1)':'Toronto', 'Boston Celtics*\xa0(2)':'Boston', 'Philadelphia 76ers*\xa0(3)':'Philadelphia', 'Cleveland Cavaliers*\xa0(4)':'Cleveland', 'Indiana Pacers*\xa0(5)':'Indianapolis', 'Miami Heat*\xa0(6)':'Miami–Fort Lauderdale', 'Milwaukee Bucks*\xa0(7)':'Milwaukee', 'Washington Wizards*\xa0(8)':'Washington, D.C.', 'Detroit Pistons\xa0(9)':'Detroit', 'Charlotte Hornets\xa0(10)':'Charlotte', 'New York Knicks\xa0(11)':'New York City', 'Brooklyn Nets\xa0(12)':'New York City', 'Chicago Bulls\xa0(13)':'Chicago', 'Orlando Magic\xa0(14)':'Orlando', 'Atlanta Hawks\xa0(15)':'Atlanta', 'Houston Rockets*\xa0(1)':'Houston', 'Golden State Warriors*\xa0(2)':'San Francisco Bay Area', 'Portland Trail Blazers*\xa0(3)':'Portland', 'Oklahoma City Thunder*\xa0(4)':'Oklahoma City', 'Utah Jazz*\xa0(5)':'Salt Lake City', 'New Orleans Pelicans*\xa0(6)':'New Orleans', 'San Antonio Spurs*\xa0(7)':'San Antonio', 'Minnesota Timberwolves*\xa0(8)':'Minneapolis–Saint Paul', 'Denver Nuggets\xa0(9)':'Denver', 'Los Angeles Clippers\xa0(10)':'Los Angeles', 'Los Angeles Lakers\xa0(11)':'Los Angeles', 'Sacramento Kings\xa0(12)':'Sacramento', 'Dallas Mavericks\xa0(13)':'Dallas–Fort Worth', 'Memphis Grizzlies\xa0(14)':'Memphis', 'Phoenix Suns\xa0(15)':'Phoenix'}, inplace=True)
nba_2018.replace('\*\s\([0-9]\)|\s\([0-9]+\)','', regex=True, inplace=True)
nba = nba_2018.filter(items=['Metropolitan area', 'W/L%'])
nba['W/L%'] = nba['W/L%'].astype(float)
nba_2= nba.groupby('Metropolitan area').agg({'W/L%':np.mean})
nba_2.sort_index(inplace=True)

def nba_correlation():
    population_by_region = list(city_2['Population (2016 est.)[8]'])
    win_loss_by_region = list(nba_2['W/L%'])

    assert len(population_by_region) == len(win_loss_by_region)
    assert len(population_by_region) == 28
    
    x = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return x[0]
print(nba_correlation())

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities_mlb = cities.filter(items=['Metropolitan area', 'Population (2016 est.)[8]', 'MLB'])
cities_mlb['MLB'].replace({'—':np.nan, '\[note\s[0-9]+\]':' '}, regex=True, inplace=True)
cities_mlb['MLB'].replace({'^\s':np.nan, '\s':''}, regex=True, inplace=True)
city_2 = cities_mlb.dropna()
city_2.set_index('Metropolitan area', inplace=True)
city_2.sort_index(inplace=True)
city_2['Population (2016 est.)[8]']=city_2['Population (2016 est.)[8]'].astype(float)

mlb_2018 = mlb_df[mlb_df['year'] > 2017]
mlb_2018.sort_values(by=['team'], inplace=True)
mlb_2018['Metropolitan area'] = mlb_2018['team']
mlb_2018.replace(
{'Arizona Diamondbacks':'Phoenix', 'Atlanta Braves':'Atlanta', 'Baltimore Orioles':'Baltimore', 'Boston Red Sox':'Boston', 'Chicago Cubs':'Chicago', 'Chicago White Sox':'Chicago', 'Cincinnati Reds':'Cincinnati', 'Cleveland Indians':'Cleveland', 'Colorado Rockies':'Denver', 'Detroit Tigers':'Detroit', 'Houston Astros':'Houston', 'Kansas City Royals':'Kansas City', 'Los Angeles Angels':'Los Angeles', 'Los Angeles Dodgers':'Los Angeles', 'Miami Marlins':'Miami–Fort Lauderdale', 'Milwaukee Brewers':'Milwaukee', 'Minnesota Twins':'Minneapolis–Saint Paul', 'New York Mets':'New York City', 'New York Yankees':'New York City', 'Oakland Athletics':'San Francisco Bay Area', 'Philadelphia Phillies':'Philadelphia', 'Pittsburgh Pirates':'Pittsburgh', 'San Diego Padres':'San Diego', 'San Francisco Giants':'San Francisco Bay Area', 'Seattle Mariners':'Seattle', 'St. Louis Cardinals':'St. Louis', 'Tampa Bay Rays':'Tampa Bay Area', 'Texas Rangers':'Dallas–Fort Worth', 'Toronto Blue Jays':'Toronto', 'Washington Nationals':'Washington, D.C.'}
, inplace=True)
mlb_2 = mlb_2018.filter(items=['Metropolitan area','W-L%'])
mlb = mlb_2.groupby('Metropolitan area').agg({'W-L%':np.mean})
mlb.sort_index(inplace=True)

def mlb_correlation():
    
    population_by_region = list(city_2['Population (2016 est.)[8]'])
    win_loss_by_region = list(mlb['W-L%'])

    assert len(population_by_region) == len(win_loss_by_region)
    assert len(population_by_region) == 26

    x = stats.pearsonr(population_by_region, win_loss_by_region)
    return x[0]
print(mlb_correlation())

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities_nfl = cities.filter(items=['Metropolitan area', 'Population (2016 est.)[8]', 'NFL'])
cities_nfl['NFL'].replace({'—|—\[note\s[0-9]+\]':np.nan, '\[note\s[0-9]+\]':' '}, regex=True, inplace=True)
cities_nfl['NFL'].replace({'^\s':np.nan, '\s':''}, regex=True, inplace=True)
city_2 = cities_nfl.dropna()
city_2.set_index('Metropolitan area', inplace=True)
city_2.sort_index(inplace=True)
city_2['Population (2016 est.)[8]']=city_2['Population (2016 est.)[8]'].astype(float)

nfl_2018 = nfl_df[nfl_df['year'] > 2017]
nfl_2 = nfl_2018.filter(items=['team', 'W-L%'])
nfl_2['Metro'] = nfl_2['team']
nfl_2.replace({'New England Patriots*':'Boston', 'Miami Dolphins':'Miami–Fort Lauderdale', 'Buffalo Bills':'Buffalo', 'New York Jets':'New York City', 'Baltimore Ravens*':'Baltimore', 'Pittsburgh Steelers':'Pittsburgh', 'Cleveland Browns':'Cleveland', 'Cincinnati Bengals':'Cincinnati', 'Houston Texans*':'Houston', 'Indianapolis Colts+':'Indianapolis', 'Tennessee Titans':'Nashville', 'Jacksonville Jaguars':'Jacksonville', 'Kansas City Chiefs*':'Kansas City', 'Los Angeles Chargers+':'Los Angeles', 'Denver Broncos':'Denver', 'Oakland Raiders':'San Francisco Bay Area', 'Dallas Cowboys*':'Dallas–Fort Worth', 'Philadelphia Eagles+':'Philadelphia', 'Washington Redskins':'Washington, D.C.', 'New York Giants':'New York City', 'Chicago Bears*':'Chicago', 'Minnesota Vikings':'Minneapolis–Saint Paul', 'Green Bay Packers':'Green Bay', 'Detroit Lions':'Detroit', 'New Orleans Saints*':'New Orleans', 'Carolina Panthers':'Charlotte', 'Atlanta Falcons':'Atlanta', 'Tampa Bay Buccaneers':'Tampa Bay Area', 'Los Angeles Rams*':'Los Angeles', 'Seattle Seahawks+':'Seattle', 'San Francisco 49ers':'San Francisco Bay Area', 'Arizona Cardinals':'Phoenix', 'AFC\s\w*':np.nan, 'NFC\s\w*':np.nan}, regex=True, inplace=True)
nfl_2.replace({'[\*\+]':''}, regex=True, inplace=True)
nfl = nfl_2.dropna()
nfl['W-L%']=nfl['W-L%'].astype(float)
nfl1 = nfl.sort_values(by=['Metro'], inplace=True)
nfl_ = nfl.groupby('Metro').agg({'W-L%':np.mean})


def nfl_correlation(): 
    cities_nfl = cities.filter(items=['Metropolitan area', 'Population (2016 est.)[8]', 'NFL'])
    cities_nfl['NFL'].replace({'—|—\[note\s[0-9]+\]':np.nan, '\[note\s[0-9]+\]':' '}, regex=True, inplace=True)
    cities_nfl['NFL'].replace({'^\s':np.nan, '\s':''}, regex=True, inplace=True)
    city_2 = cities_nfl.dropna()
    city_2.set_index('Metropolitan area', inplace=True)
    city_2.sort_index(inplace=True)
    city_2['Population (2016 est.)[8]']=city_2['Population (2016 est.)[8]'].astype(float)

    nfl_2018 = nfl_df[nfl_df['year'] > 2017]
    nfl_2 = nfl_2018.filter(items=['team', 'W-L%'])
    nfl_2['Metro'] = nfl_2['team']
    nfl_2.replace({'New England Patriots*':'Boston', 'Miami Dolphins':'Miami–Fort Lauderdale', 'Buffalo Bills':'Buffalo', 'New York Jets':'New York City', 'Baltimore Ravens*':'Baltimore', 'Pittsburgh Steelers':'Pittsburgh', 'Cleveland Browns':'Cleveland', 'Cincinnati Bengals':'Cincinnati', 'Houston Texans*':'Houston', 'Indianapolis Colts+':'Indianapolis', 'Tennessee Titans':'Nashville', 'Jacksonville Jaguars':'Jacksonville', 'Kansas City Chiefs*':'Kansas City', 'Los Angeles Chargers+':'Los Angeles', 'Denver Broncos':'Denver', 'Oakland Raiders':'San Francisco Bay Area', 'Dallas Cowboys*':'Dallas–Fort Worth', 'Philadelphia Eagles+':'Philadelphia', 'Washington Redskins':'Washington, D.C.', 'New York Giants':'New York City', 'Chicago Bears*':'Chicago', 'Minnesota Vikings':'Minneapolis–Saint Paul', 'Green Bay Packers':'Green Bay', 'Detroit Lions':'Detroit', 'New Orleans Saints*':'New Orleans', 'Carolina Panthers':'Charlotte', 'Atlanta Falcons':'Atlanta', 'Tampa Bay Buccaneers':'Tampa Bay Area', 'Los Angeles Rams*':'Los Angeles', 'Seattle Seahawks+':'Seattle', 'San Francisco 49ers':'San Francisco Bay Area', 'Arizona Cardinals':'Phoenix', 'AFC\s\w*':np.nan, 'NFC\s\w*':np.nan}, regex=True, inplace=True)
    nfl_2.replace({'[\*\+]':''}, regex=True, inplace=True)
    nfl = nfl_2.dropna()
    nfl['W-L%']=nfl['W-L%'].astype(float)
    nfl1 = nfl.sort_values(by=['Metro'], inplace=True)
    nfl_ = nfl.groupby('Metro').agg({'W-L%':np.mean})
    
    population_by_region = list(city_2['Population (2016 est.)[8]']) # pass in metropolitan area population from cities
    win_loss_by_region = list(nfl_['W-L%']) # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    x = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return x[0]
nfl_correlation()

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


nhl_2018 = nhl_df[nhl_df['year'] > 2017]
cities_nhl = cities.filter(items=['Metropolitan area', 'Population (2016 est.)[8]', 'NHL'])
cities_nhl['NHL'].replace({'—':np.nan, '\[note\s[0-9]+\]':' '}, regex=True, inplace=True)
cities_nhl['NHL'].replace({'^\s':np.nan, '\s':''}, regex=True, inplace=True)
city_2 = cities_nhl.dropna()
city_2.set_index('Metropolitan area', inplace=True)
city_2.sort_index(inplace=True)
city_2['Population (2016 est.)[8]']=city_2['Population (2016 est.)[8]'].astype(int)
nhl_2018['Metropolitan area']=nhl_2018['team']
nhl_2018['Metropolitan area'].replace({'Tampa Bay Lightning*':'Tampa Bay Area', 'Boston Bruins*':'Boston', 'Toronto Maple Leafs*':'Toronto', 'Florida Panthers':'Miami–Fort Lauderdale', 'Detroit Red Wings':'Detroit', 'Montreal Canadiens':'Montreal', 'Ottawa Senators':'Ottawa', 'Buffalo Sabres':'Buffalo','Washington Capitals*':'Washington, D.C.', 'Pittsburgh Penguins*':'Pittsburgh', 'Philadelphia Flyers*':'Philadelphia', 'Columbus Blue Jackets*':'Columbus', 'New Jersey Devils*':'New York City', 'Carolina Hurricanes':'Raleigh', 'New York Islanders':'New York City', 'New York Rangers':'New York City', 'Nashville Predators*':'Nashville', 'Winnipeg Jets*':'Winnipeg', 'Minnesota Wild*':'Minneapolis–Saint Paul', 'Colorado Avalanche*':'Denver', 'St. Louis Blues':'St. Louis', 'Dallas Stars':'Dallas–Fort Worth', 'Chicago Blackhawks':'Chicago','Vegas Golden Knights*':'Las Vegas', 'Anaheim Ducks*':'Los Angeles', 'San Jose Sharks*':'San Francisco Bay Area', 'Los Angeles Kings*':'Los Angeles', 'Calgary Flames':'Calgary', 'Edmonton Oilers':'Edmonton', 'Vancouver Canucks':'Vancouver', 'Arizona Coyotes':'Phoenix'}, inplace=True)
nhl_2018.replace('\w*\sDivision',np.nan, regex=True, inplace=True)
nhl = nhl_2018.dropna().filter(items=['W', 'L', 'Metropolitan area'])
nhl[['W', 'L']] = nhl[['W', 'L']].astype(int)
nhl_2= nhl.groupby('Metropolitan area').agg({'W':np.mean, 'L':np.mean})
nhl_2['W/L ratio'] = nhl_2['W']/(nhl_2['W']+nhl_2['L'])
nhl_2.drop(columns=['W', 'L'], axis=1, inplace=True)

cities_nba = cities.filter(items=['Metropolitan area', 'Population (2016 est.)[8]', 'NBA'])
cities_nba['NBA'].replace({'—':np.nan, '\[note\s[0-9]+\]':' '}, regex=True, inplace=True)
cities_nba['NBA'].replace({'^\s':np.nan, '\s':''}, regex=True, inplace=True)
city_2 = cities_nba.dropna()
city_2.set_index('Metropolitan area', inplace=True)
city_2.sort_index(inplace=True)
city_2['Population (2016 est.)[8]']=city_2['Population (2016 est.)[8]'].astype(float)
nba_2018 = nba_df[nba_df['year'] > 2017]
nba_2018['Metropolitan area']=nba_2018['team']
nba_2018['Metropolitan area'].replace({'Toronto Raptors*\xa0(1)':'Toronto', 'Boston Celtics*\xa0(2)':'Boston', 'Philadelphia 76ers*\xa0(3)':'Philadelphia', 'Cleveland Cavaliers*\xa0(4)':'Cleveland', 'Indiana Pacers*\xa0(5)':'Indianapolis', 'Miami Heat*\xa0(6)':'Miami–Fort Lauderdale', 'Milwaukee Bucks*\xa0(7)':'Milwaukee', 'Washington Wizards*\xa0(8)':'Washington, D.C.', 'Detroit Pistons\xa0(9)':'Detroit', 'Charlotte Hornets\xa0(10)':'Charlotte', 'New York Knicks\xa0(11)':'New York City', 'Brooklyn Nets\xa0(12)':'New York City', 'Chicago Bulls\xa0(13)':'Chicago', 'Orlando Magic\xa0(14)':'Orlando', 'Atlanta Hawks\xa0(15)':'Atlanta', 'Houston Rockets*\xa0(1)':'Houston', 'Golden State Warriors*\xa0(2)':'San Francisco Bay Area', 'Portland Trail Blazers*\xa0(3)':'Portland', 'Oklahoma City Thunder*\xa0(4)':'Oklahoma City', 'Utah Jazz*\xa0(5)':'Salt Lake City', 'New Orleans Pelicans*\xa0(6)':'New Orleans', 'San Antonio Spurs*\xa0(7)':'San Antonio', 'Minnesota Timberwolves*\xa0(8)':'Minneapolis–Saint Paul', 'Denver Nuggets\xa0(9)':'Denver', 'Los Angeles Clippers\xa0(10)':'Los Angeles', 'Los Angeles Lakers\xa0(11)':'Los Angeles', 'Sacramento Kings\xa0(12)':'Sacramento', 'Dallas Mavericks\xa0(13)':'Dallas–Fort Worth', 'Memphis Grizzlies\xa0(14)':'Memphis', 'Phoenix Suns\xa0(15)':'Phoenix'}, inplace=True)
nba_2018.replace('\*\s\([0-9]\)|\s\([0-9]+\)','', regex=True, inplace=True)
nba = nba_2018.filter(items=['Metropolitan area', 'W/L%'])
nba['W/L%'] = nba['W/L%'].astype(float)
nba_2= nba.groupby('Metropolitan area').agg({'W/L%':np.mean})
nba_2.sort_index(inplace=True)



cities_mlb = cities.filter(items=['Metropolitan area', 'Population (2016 est.)[8]', 'MLB'])
cities_mlb['MLB'].replace({'—':np.nan, '\[note\s[0-9]+\]':' '}, regex=True, inplace=True)
cities_mlb['MLB'].replace({'^\s':np.nan, '\s':''}, regex=True, inplace=True)
city_2 = cities_mlb.dropna()
city_2.set_index('Metropolitan area', inplace=True)
city_2.sort_index(inplace=True)
city_2['Population (2016 est.)[8]']=city_2['Population (2016 est.)[8]'].astype(float)
mlb_2018 = mlb_df[mlb_df['year'] > 2017]
mlb_2018.sort_values(by=['team'], inplace=True)
mlb_2018['Metropolitan area'] = mlb_2018['team']
mlb_2018.replace(
{'Arizona Diamondbacks':'Phoenix', 'Atlanta Braves':'Atlanta', 'Baltimore Orioles':'Baltimore', 'Boston Red Sox':'Boston', 'Chicago Cubs':'Chicago', 'Chicago White Sox':'Chicago', 'Cincinnati Reds':'Cincinnati', 'Cleveland Indians':'Cleveland', 'Colorado Rockies':'Denver', 'Detroit Tigers':'Detroit', 'Houston Astros':'Houston', 'Kansas City Royals':'Kansas City', 'Los Angeles Angels':'Los Angeles', 'Los Angeles Dodgers':'Los Angeles', 'Miami Marlins':'Miami–Fort Lauderdale', 'Milwaukee Brewers':'Milwaukee', 'Minnesota Twins':'Minneapolis–Saint Paul', 'New York Mets':'New York City', 'New York Yankees':'New York City', 'Oakland Athletics':'San Francisco Bay Area', 'Philadelphia Phillies':'Philadelphia', 'Pittsburgh Pirates':'Pittsburgh', 'San Diego Padres':'San Diego', 'San Francisco Giants':'San Francisco Bay Area', 'Seattle Mariners':'Seattle', 'St. Louis Cardinals':'St. Louis', 'Tampa Bay Rays':'Tampa Bay Area', 'Texas Rangers':'Dallas–Fort Worth', 'Toronto Blue Jays':'Toronto', 'Washington Nationals':'Washington, D.C.'}
, inplace=True)
mlb_2 = mlb_2018.filter(items=['Metropolitan area','W-L%'])
mlb = mlb_2.groupby('Metropolitan area').agg({'W-L%':np.mean})
mlb.sort_index(inplace=True)




cities_nfl = cities.filter(items=['Metropolitan area', 'Population (2016 est.)[8]', 'NFL'])
cities_nfl['NFL'].replace({'—|—\[note\s[0-9]+\]':np.nan, '\[note\s[0-9]+\]':' '}, regex=True, inplace=True)
cities_nfl['NFL'].replace({'^\s':np.nan, '\s':''}, regex=True, inplace=True)
city_2 = cities_nfl.dropna()
city_2.set_index('Metropolitan area', inplace=True)
city_2.sort_index(inplace=True)
city_2['Population (2016 est.)[8]']=city_2['Population (2016 est.)[8]'].astype(float)
nfl_2018 = nfl_df[nfl_df['year'] > 2017]
nfl_2 = nfl_2018.filter(items=['team', 'W-L%']).rename(columns={'W-L%':'W_L%'})
nfl_2['Metropolitan area'] = nfl_2['team']
nfl_2.replace({'New England Patriots*':'Boston', 'Miami Dolphins':'Miami–Fort Lauderdale', 'Buffalo Bills':'Buffalo', 'New York Jets':'New York City', 'Baltimore Ravens*':'Baltimore', 'Pittsburgh Steelers':'Pittsburgh', 'Cleveland Browns':'Cleveland', 'Cincinnati Bengals':'Cincinnati', 'Houston Texans*':'Houston', 'Indianapolis Colts+':'Indianapolis', 'Tennessee Titans':'Nashville', 'Jacksonville Jaguars':'Jacksonville', 'Kansas City Chiefs*':'Kansas City', 'Los Angeles Chargers+':'Los Angeles', 'Denver Broncos':'Denver', 'Oakland Raiders':'San Francisco Bay Area', 'Dallas Cowboys*':'Dallas–Fort Worth', 'Philadelphia Eagles+':'Philadelphia', 'Washington Redskins':'Washington, D.C.', 'New York Giants':'New York City', 'Chicago Bears*':'Chicago', 'Minnesota Vikings':'Minneapolis–Saint Paul', 'Green Bay Packers':'Green Bay', 'Detroit Lions':'Detroit', 'New Orleans Saints*':'New Orleans', 'Carolina Panthers':'Charlotte', 'Atlanta Falcons':'Atlanta', 'Tampa Bay Buccaneers':'Tampa Bay Area', 'Los Angeles Rams*':'Los Angeles', 'Seattle Seahawks+':'Seattle', 'San Francisco 49ers':'San Francisco Bay Area', 'Arizona Cardinals':'Phoenix', 'AFC\s\w*':np.nan, 'NFC\s\w*':np.nan}, regex=True, inplace=True)
nfl_2.replace({'[\*\+]':''}, regex=True, inplace=True)
nfl = nfl_2.dropna()
nfl['W_L%']=nfl['W_L%'].astype(float)
nfl1 = nfl.sort_values(by=['Metropolitan area'], inplace=True)
nfl_ = nfl.groupby('Metropolitan area').agg({'W_L%':np.mean})


merged1 = pd.merge(nhl_2, nba_2, how='inner', left_index=True, right_index=True)
m1_stat = stats.ttest_rel(merged1['W/L ratio'], merged1['W/L%'])

merged2 = pd.merge(nhl_2, mlb, how='inner', left_index=True, right_index=True)
m2_stat = stats.ttest_rel(merged2['W/L ratio'], merged2['W-L%'])

merged3 = pd.merge(nhl_2, nfl_, how='inner', left_index=True, right_index=True)
m3_stat = stats.ttest_rel(merged3['W/L ratio'], merged3['W_L%'])

merged4 = pd.merge(nba_2, mlb, how='inner', left_index=True, right_index=True)
m4_stat = stats.ttest_rel(merged4['W/L%'], merged4['W-L%'])

merged5 = pd.merge(nba_2, nfl_, how='inner', left_index=True, right_index=True)
m5_stat = stats.ttest_rel(merged5['W/L%'], merged5['W_L%'])

merged6 = pd.merge(mlb, nfl_, how='inner', left_index=True, right_index=True)
m6_stat = stats.ttest_rel(merged6['W-L%'], merged6['W_L%'])

def sports_team_performance():
    
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    p_values.at['NHL', 'NBA']=m1_stat[1]
    p_values.at['NHL', 'MLB']=m2_stat[1]
    p_values.at['NHL', 'NFL']=m3_stat[1]
    p_values.at['NBA', 'MLB']=m4_stat[1]
    p_values.at['NBA', 'NFL']=m5_stat[1]
    p_values.at['MLB', 'NFL']=m6_stat[1]
    
    p_values.at['NBA', 'NHL']=m1_stat[1]
    p_values.at['MLB', 'NHL']=m2_stat[1]
    p_values.at['NFL', 'NHL']=m3_stat[1]
    p_values.at['MLB', 'NBA']=m4_stat[1]
    p_values.at['NFL', 'NBA']=m5_stat[1]
    p_values.at['NFL', 'MLB']=m6_stat[1]
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
sports_team_performance()